In [1]:
import os
import time

import aprofiles as apro
import numpy as np
import xarray as xr

start_time = time.time()

BASE_DIR_OUT = 'data/v-profiles'
date = '2021-09-09'
yyyy = date.split('-')[0]
mm = date.split('-')[1]
dd = date.split('-')[2]

In [2]:
datepath = os.path.join(BASE_DIR_OUT,yyyy,mm,dd)
onlyfiles = [f for f in os.listdir(datepath) if os.path.isfile(os.path.join(datepath, f))]

In [3]:
filename = onlyfiles[0]
ds = xr.open_dataset(os.path.join(datepath, filename), decode_times=True)

In [11]:
scene_classes = ['low_cloud', 'mid_cloud', 'high_cloud', 'foc', 'aer']

counts = {}
for scene_class in scene_classes:
    counts[scene_class] = len(ds.scene.data[ds.scene.data == scene_class])
counts['total'] = len(ds.scene.data)

In [31]:
ds = ds.assign_coords(time=ds.time.data.astype("datetime64[{}]".format(resolution)).astype(int))
ds["time"] = ds["time"].assign_attrs(time_attrs)
ds["time"].attrs['units'] = 'milliseconds after epoch'

NameError: name 'resolution' is not defined

In [91]:
# needs to convert time from int to date in order to use the resample method
ds = ds.assign_coords(time = ds.time.data.astype("datetime64[ms]"))
max_ext_profiles = ds.extinction.resample(time="1H").max().data

# take the maximum value in each profile
max_ext = np.nanmax(max_ext_profiles, axis=1)

# attribute a weight to each scene in order to prioritize the scenes
scene_weights = {'foc': 4, 'low_cloud': 3, 'mid_cloud': 2, 'high_cloud': 1, 'aer': 0}

scene = np.asarray(ds.scene.data)
for scene_weight in scene_weights.keys():
    print(scene_weight)
    scene[scene==scene_weight] = scene_weights[scene_weight]

# creates dataarrays
ds["weight_scene"] = ('time', scene)
# takes the highest weight for resampled data
max_weight = ds.scene.resample(time="1H").max().data
avg_scene = []

# reverse dictionnary
weight_scenes = {v: k for k, v in scene_weights.items()}
max_scenes = max_weight
for weight_scene in weight_scenes.keys():
    max_scenes[max_scenes==weight_scene] = weight_scenes[weight_scene]


foc
low_cloud
mid_cloud
high_cloud
aer


<ipython-input-91-2be334bae513>:6: RuntimeWarning: All-NaN slice encountered
  max_ext = np.nanmax(max_ext_profiles, axis=1)


In [80]:
# creates dataarrays
ds["weight_scene"] = ('time', scene)
# takes the highest weight for resampled data
max_weight = ds.int_scene.resample(time="1H").max().data
avg_scene = []

# reverse dictionnary
weight_scenes = {v: k for k, v in scene_weights.items()}
max_scenes = max_weight
for weight_scene in weight_scenes.keys():
    max_scenes[max_scenes==weight_scene] = weight_scenes[weight_scene]